In [1]:
## Use conda
## conda create --yes -n snorkel-env python=3.6
## conda activate snorkel-env

# We specify PyTorch here to ensure compatibility, but it may not be necessary.
## conda install pytorch==1.1.0 -c pytorch
## conda install snorkel==0.9.0 -c conda-forge
## conda install -c conda-forge jupyterlab
import snorkel

In [2]:
import pandas as pd

In [3]:
crowd_df_1 =pd.read_csv('../data/crowd/match_crowd_data.csv')
crowd_df_1.sample()

,Unnamed: 0,DB_ID,DO_ID,disease,Label_ID,relation,Section,Set_ID,Drug_Brand_Name,Active_ingredient,UNII_ID,Context,WordCount,Drug_name
1384,3205,DB00232,DOID_2983,anuria,41d6b92e-b9f4-4bcf-9f66-9a36c8e79181.xml,relief,Contraindications,4910dad2-c533-4232-85bf-e6b0235379c2,Methyclothiazide,METHYCLOTHIAZIDE,L3H46UAC61,Methyclothiazide tablets are contraindicated i...,23,Methyclothiazide


In [4]:

crowd_df_1 = crowd_df_1[['DB_ID', 'DO_ID','disease', 'Label_ID','Set_ID', 'relation', 'Drug_name', 'Context','Section' ]]
crowd_df_1.sample()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
1334,DB09080,DOID_2841,asthma,395efad5-f92f-4958-dbc9-deae88e9e032.xml,5d9a5433-857e-44ea-b3d6-2a400ca0cef5,contraindication,Olodaterol,All LABA are contraindicated in patients with ...,Contraindications


In [5]:
crowd_df_plus = pd.read_csv('../data/crowd/crowd_2nd.csv')
crowd_df_plus = crowd_df_plus[['DB_ID', 'DO_ID','disease', 'Label_ID', 'Set_ID','relation', 'Drug_name', 'Context','Section' ]]
crowd_df_plus.sample()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
105,DB00959,DOID_2377,multiple sclerosis,2b544c6a-963c-4054-8eb1-6f093004efe7.xml,9ff2ec25-57b5-4466-b51f-1d4bb924e34f,relief,METHYLPREDNISOLONE,"Methylprednisolone tablets, USP are indicated ...",Indications


In [6]:

crowd_df = crowd_df_1.append(crowd_df_plus)
len(crowd_df)

2185

In [7]:
crowd_df= crowd_df.drop_duplicates(subset=['Label_ID','DO_ID','Section'], keep='last')
len(crowd_df)

2162

In [8]:
unlabeled_df  = pd.read_csv('../data/unlabeled/raw_cleaned.csv')
unlabeled_df = unlabeled_df[['DB_ID', 'DO_ID','disease', 'Label_ID', 'DrugName', 'Context','Section' ]]
len(unlabeled_df)

54040

In [9]:
unlabeled_df= unlabeled_df.rename(columns={'DrugName':'Drug_name'})

In [10]:
unlabeled_df.drop_duplicates(subset=['Context','DO_ID'], keep='first', inplace=True)
len(unlabeled_df)

39810

In [11]:
experts = pd.read_csv('../data/expert/clean_expert_resolved.csv')
experts = experts[['DB_ID', 'DO_ID','disease', 'Label_ID', 'Set_ID','relation', 'Drug_name', 'Context','Section']]
experts.sample()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
116,DB00461,DOID_7148,RHEUMATOID ARTHRITIS,a34aebce-f0b6-4341-8c7f-49bdd1c06fe9.xml,c3f72a74-1966-4a68-bd2d-692d41d05066,Indication: Symptomatic Relief,Nabumetone,Carefully consider the potential benefits and ...,Indications


In [12]:
len(experts)

284

In [13]:
def rep(text):
    a = text.replace('Contraindication', 'contraindication')
    b = a.replace('Indication: Treatment', 'treatment')
    c = b.replace('Indication: Symptomatic Relief', 'relief')
    d = c.replace('Effect', 'effect')
    return d

def lower (x):
    return x.lower()

# remove punctuation
def remove_symbols(x):
    for char in x:
            if char not in ['1','2','3','4','5','6','7','8','9','q','Q','w','W', 'e','E','r','R','t','T','y','Y','U','u','i','I','O','o','P','p','A','a','S','s','D','d','F','f','G','g','H','h','J','j','K','k','L','l','Z','z','X','x','C','c','V','v','B','b','N','n','M','m']:
                x = x.replace(char, ' ')
    return x

def check(x, text):
    return x in text

def remove_xml_symbols(text):
    text1 = text.replace('\n', '  ')
    clean_text = text1.replace('\r', '  ')
    return clean_text

def remove_spaces(text):
    clean_text = ' '.join(text.split())
    return clean_text

experts['relation'] =experts['relation'].apply(rep)

experts['Context'] = experts['Context'].apply(remove_xml_symbols)
experts['Context'] = experts['Context'].apply(remove_symbols)
experts['Context'] = experts['Context'].apply(remove_spaces)
experts['Context'] = experts['Context'].apply(lower)

experts['disease'] = experts['disease'].apply(remove_xml_symbols)
experts['disease'] = experts['disease'].apply(remove_symbols)
experts['disease'] = experts['disease'].apply(remove_spaces)
experts['disease'] = experts['disease'].apply(lower)


In [14]:
crowd_df['Context'] = crowd_df['Context'].apply(remove_xml_symbols)
crowd_df['Context'] = crowd_df['Context'].apply(remove_symbols)
crowd_df['Context'] = crowd_df['Context'].apply(remove_spaces)
crowd_df['Context'] = crowd_df['Context'].apply(lower)

crowd_df['disease'] = crowd_df['disease'].apply(remove_xml_symbols)
crowd_df['disease'] = crowd_df['disease'].apply(remove_symbols)
crowd_df['disease'] = crowd_df['disease'].apply(remove_spaces)
crowd_df['disease'] = crowd_df['disease'].apply(lower)



In [15]:
crowd_df.sample(2)

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
496,DB00669,DOID_3393,coronary artery disease,c7b7632b-aaaa-41d3-9b3f-dd59e0d4c0e3.xml,a1b60c11-9c87-4400-86dd-2c28284298d9,contraindication,SUMATRIPTAN,sumatriptan tablets are contraindicated in pat...,Contraindication
616,DB06402,DOID_874,bacterial pneumonia,848f842a-bac0-4694-8705-24f8ba5a90fc.xml,c09655ea-7af5-4443-a769-86e416214f59,treatment,Telavancin,vibativ is a lipoglycopeptide antibacterial dr...,Indications


In [16]:
test_df = experts
test_df.sample(4)

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
283,DB01142,DOID_1686,glaucoma,5205d855-ccf4-4d3f-be11-bda7476a62a3.xml,1bec1223-5239-4eb6-a9e8-62444106d2c0,contraindication,Doxepin,hypersensitivity to doxepin hydrochloride inac...,Contraindications
272,DB00696,DOID_3393,coronary heart disease,6ac058ea-d4f4-4fcd-83e1-5f2adb8652e6.xml,1bfd4827-5123-4dbb-8f94-7b097f5bfd5c,contraindication,Ergotamine,coadministration of ergotamine with potent cyp...,Contraindications
237,DB09462,DOID_417,autoimmune disease,fb54abb5-85cd-485f-9542-a2742a31bf72.xml,9c75abf7-b941-4615-b059-bcc41c64d612,contraindication,Glycerin,do not administer in the presence of diseases ...,Contraindications
215,DB00951,DOID_848,arthritis,75907d04-0c9d-4a6a-b8c0-7d88ab5ad337.xml,db8a3ae7-fb5b-47a9-a9f7-f57cd90501c2,effect,Isoniazid,isoniazid is contraindicated in patients who d...,Contraindications


In [17]:
experts = experts.drop_duplicates(subset=['Context','DO_ID'], keep='last')
len(experts)

282

In [18]:
on_col =['DO_ID', 'Label_ID', 'Set_ID','Section']


In [19]:
train_df = crowd_df[~crowd_df[on_col].apply(tuple,1).isin(experts[on_col].apply(tuple,1))]
len(train_df)

1881

In [20]:
test_df =crowd_df.merge(experts[on_col], on=on_col)
len(test_df)

281

In [21]:
test_df.head()

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
0,DB00657,DOID_10824,malignant hypertension,976f404f-4857-4bad-9a3c-9bf3d5ca9f5b.xml,c6822ce4-e741-4366-b72a-5f5146354f97,treatment,Mecamylamine,for the management of moderately severe to sev...,Indications
1,DB00401,DOID_10763,hypertension,ce733b39-7857-4538-92d6-1c68a4e4eb75.xml,a6a8a8ba-3966-4f37-a58b-44d3b05bdc4e,treatment,Nisoldipine,nisoldipine extended release tablets are indic...,Indications
2,DB00612,DOID_10763,hypertension,c818c38d-ee37-4ceb-b078-611483b4f743.xml,c818c38d-ee37-4ceb-b078-611483b4f743,treatment,Bisoprolol,bisoprolol fumarate tablets usp are indicated ...,Indications
3,DB00960,DOID_10763,hypertension,d4078b63-30ad-435b-a081-4d90562962b5.xml,b5a5a1e9-ee9b-41b1-b2d5-5f5834b816a7,treatment,Pindolol,pindolol tablets are indicated in the manageme...,Indications
4,DB00876,DOID_10763,hypertension,ab531d28-345b-4113-9fde-00cd63ad5b1a.xml,64fbc500-3b77-4c9c-8576-b3ec5f6655a5,treatment,Eprosartan,eprosartan mesylate tablets are indicated for ...,Indications


In [22]:
train_df =unlabeled_df.append(train_df)
print (len(train_df))
train_df.drop_duplicates(subset=['Context','DO_ID'], keep='last', inplace=True)
print (len(train_df))

41691
40516


/Users/remzicelebi/opt/anaconda3/envs/snorkel-env/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [23]:
dev_df = test_df

In [24]:
import numpy as np
indx = dev_df.sample(140).index
dev_df.loc[indx,'relation'] = np.nan
#temp_df['relation'] = 

In [25]:
dev_df.sample(10)

,DB_ID,DO_ID,disease,Label_ID,Set_ID,relation,Drug_name,Context,Section
190,DB00986,DOID_1686,glaucoma,d0729857-b70a-4f86-8c5e-3b5b96e6123e.xml,d200bd44-9856-4104-a29e-a4cca3db6737,NaN,Glycopyrronium,cuvposa is contraindicated in patients with me...,Contraindications
271,DB01579,DOID_1686,glaucoma,281377de-81c3-4fec-96e9-7b426190c3f9.xml,34682387-2323-4b66-b166-59aec55f45f5,NaN,Phendimetrazine,known hypersensitivity or idiosyncratic reacti...,Contraindications
191,DB00202,DOID_0060319,cardiac arrest,f03f24d8-b519-44b2-bfd1-a9e91498448e.xml,c06156ae-b889-4250-bc85-1435b2635829,NaN,Succinylcholine,succinylcholine is contraindicated in persons ...,Contraindications
35,DB00457,DOID_10763,hypertension,e64473aa-3115-46d0-94d0-362e48e6ad38.xml,35c7aa71-6171-4ba5-916c-384c28e5f5c1,treatment,Prazosin,prazosin hydrochloride capsules usp is indicat...,Indications
250,DB00880,DOID_2983,anuria,1dab3243-f1dc-467d-b341-72aea14a0cce.xml,5046a4d7-7742-4f19-930a-2582cb41474e,NaN,Chlorothiazide,anuria hypersensitivity to any component of th...,Contraindications
154,DB00307,DOID_0060058,lymphoma,8102cbe3-c43e-4e49-bd13-2acbe511c093.xml,d7fc7c7b-2ecf-4543-8bb4-9b4415d66613,NaN,Bexarotene,bexarotene capsules are indicated for the trea...,Indications
123,DB00749,DOID_7148,rheumatoid arthritis,8bc073da-7e01-4e0f-91bf-e5440b31bd1f.xml,0c5f8631-f013-4bbb-b610-0a8a1311844e,relief,Etodolac,carefully consider the potential benefits and ...,Indications
93,DB00734,DOID_5419,schizophrenia,c031d46a-1f80-4600-ab2e-ff0b6e267e2a.xml,f0c765e9-edc1-4944-a4e6-91b82a29a8d7,treatment,Risperidone,enter section text here treatment of schizophr...,Indications
235,DB00524,DOID_2983,anuria,20d39fb1-12c2-4010-8010-ef55d8f48a08.xml,78105e5c-5d36-426c-9f79-f2e085460b75,contraindication,Metolazone,anuria hepatic coma or precoma known allergy o...,Contraindications
218,DB00367,DOID_10616,acute cervicitis,56207206-c9df-41bd-beda-58a97e4a8624.xml,aaf0eb2a-f88a-4f26-a445-0fd30176c326,NaN,Levonorgestrel,the use of liletta is contraindicated when one...,Contraindications


In [26]:
len(train_df)

40516

In [27]:
dev_df.merge(experts, on=['Label_ID','DO_ID'])[['relation_x','relation_y']]

,relation_x,relation_y
0,NaN,treatment
1,treatment,treatment
2,NaN,treatment
3,treatment,treatment
4,NaN,treatment
...,...,...
276,NaN,contraindication
277,effect,contraindication
278,NaN,contraindication
279,relief,contraindication


In [28]:
Y_dev = dev_df.merge(experts, on=['Label_ID','DO_ID'])['relation_y']

In [29]:
classes_y = {
'effect': 0,
'contraindication': 1,
'relief': 2,
'treatment':3 
}
Y_dev =Y_dev.map(classes_y)

In [30]:
from snorkel.labeling import LFAnalysis

In [31]:
my_phrases = { 'effect': 
["hypersensitivity reactions",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning" 
],

 'contraindication': 
["not administrated to",
"contraindicated in",
"contraindicatedin",
"should not be used",
"is contraindication for",
"is contraindication when",
"is contraindicated when",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use"],
              
 'relief': 
["relief of the signs",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"], 

 'treatment':
[" indicated for the treatment of",
" indicated in the management of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of",
" indicated as an adjunct"]
           }

phrases = pd.DataFrame({ key:pd.Series(value) for key, value in my_phrases.items() })

effect = { 'phrases': 
["hypersensitivity reactions",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning" 
]}
effect_df = pd.DataFrame(effect)

 
contraindication = {'phrases': 
["not administrated to",
"contraindicated in",
"contraindicatedin",
"should not be used",
"is contraindication for",
"is contraindication when",
"is contraindicated when",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use"]}
contraindication_df = pd.DataFrame(contraindication)
              
relief = {'phrases': 
["relief of the signs",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"]} 
relief_df = pd.DataFrame(relief)

treatment = {'phrases':
[" indicated for the treatment of",
" indicated in the management of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of"]}
treatment_df = pd.DataFrame(treatment)

In [32]:
from snorkel.labeling import labeling_function
from snorkel.preprocess import preprocessor


def check(x, string):
    return x in string


def find_phrases(text):
    # Find which phrases appear in the sentence
    class_effect = effect_df['phrases'].apply(lambda num : check(num,text))
    class_contraindication = contraindication_df['phrases'].apply(lambda num : check(num,text))
    class_relief = relief_df['phrases'].apply(lambda num : check(num,text))
    class_treatment = treatment_df['phrases'].apply(lambda num : check(num,text))
    
    #Create a dataset from them
    classes = {
    'effect': class_effect,
    'contraindication': class_contraindication,
    'relief': class_relief,
    'treatment':class_treatment 
    }

    classes = pd.DataFrame(classes)
    
    a = class_effect.index[class_effect == True].tolist()
    b = class_contraindication.index[class_contraindication == True].tolist()
    c = class_relief.index[class_relief == True].tolist()
    d = class_treatment.index[class_treatment == True].tolist()
    return classes, a, b, c, d
                                                 

# return the position of a phrase/word in a sentence
def pos_of_phrase(string, substring):
    substring_length = len(substring)    
    def recurse(locations_found, start):
        location = string.find(substring, start)
        if location != -1:
            return recurse(locations_found + [location], location+substring_length)
        else:
            return locations_found

    return recurse([], 0)

# return the minimum distance of all phrase/word from the target disease
def get_diff_of_pos(classes, column, pos, string):
    name = classes.columns[column]
    my_col = classes[name]
    x = [k for k,v in enumerate(my_col) if v == True]
    
    candidate = []
    distances = []
    for i in range(len(x)):
        word = phrases[classes.columns[column]][x[i]]
        
        for i in range(len(pos_of_phrase(string, word))):
            distances.append( abs(pos -   pos_of_phrase(string, word)[i]) )

    return min(distances)


@labeling_function()
def phrases_claas(x):
    
    text = x.Context

    classes, a, b, c, d =find_phrases(text)


    # Check if you have unique class and label based on that 
    if len(a)!=0 and len(b)==len(c)==len(d)==0:
        #print('effect')
        return RELATION_CLASSES['effect']
    elif len(b)!=0 and len(a)==len(c)==len(d)==0:
        #print('contraindication')
        return RELATION_CLASSES['contraindication']
    elif len(c)!=0 and len(a)==len(b)==len(d)==0:
        #print('relief')
        return  RELATION_CLASSES['relief']
    elif len(d)!=0 and len(a)==len(b)==len(c)==0:
        #print('treatment')
        return  RELATION_CLASSES['treatment']
    elif len(a)==len(b)==len(c)==len(d)==0:
        #print('IDK')
        return ABSTAIN
    return ABSTAIN

@labeling_function()
def phrase_closeness(x):
    text = x.Context
    pos1 = text.find(x.disease)
    dis_len = len(x.disease)
    classes, a, b, c, d = find_phrases(text)
    
    minDist = len(text.split())
    category = ABSTAIN
    for class_,found_phrases in {'effect':a, 'contraindication':b, 'relief':c, 'treatment':d}.items():
        for phrase_index in found_phrases:
            phrase =phrases.loc[phrase_index,class_]
            pos2 = text.find(phrase)
            phr_len = len(phrase)
            if pos1 > pos2:
                span = text[pos2+phr_len:pos1]
            else:
                span = text[pos1+dis_len:pos2]
            words = span.split()
            distance = len(words)
            if minDist > distance:
                minDist = distance
                category = class_
    
    if category == ABSTAIN:
        return ABSTAIN
    return RELATION_CLASSES[category]

@labeling_function()
def crowd(x):
    #print (x.relation_type)
    if x.relation in RELATION_CLASSES:
        return RELATION_CLASSES[x.relation]
    return ABSTAIN


In [33]:
ABSTAIN = -1

RELATION_CLASSES = {
'effect': 0,
'contraindication': 1,
'relief': 2,
'treatment':3 
} 
RELATION_CLASSES

{'effect': 0, 'contraindication': 1, 'relief': 2, 'treatment': 3}

In [34]:
from snorkel.labeling import PandasLFApplier

lfs = [crowd, phrases_claas, phrase_closeness]
applier = PandasLFApplier(lfs)

In [35]:
L_train = applier.apply(train_df)

100%|██████████| 40516/40516 [04:44<00:00, 142.32it/s]


In [36]:
L_dev = applier.apply(dev_df)

100%|██████████| 281/281 [00:02<00:00, 127.40it/s]


In [37]:
Y_dev.unique()

array([3, 0, 2, 1])

In [38]:
from snorkel.labeling import LabelModel
# Train LabelModel.
label_model = LabelModel(cardinality=4, verbose=True)
label_model.fit(L_train, n_epochs=100, seed=123, log_freq=20, l2=0.1, lr=0.01)

In [41]:
from snorkel.analysis import metric_score

preds_dev = label_model.predict(L_dev)

In [44]:
acc = metric_score(Y_dev, preds_dev, probs=None, metric="accuracy")
acc

0.8149466192170819

In [45]:
LFAnalysis(L_dev, lfs).lf_summary(Y_dev.values).head()

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
crowd,0,"[0, 1, 2, 3]",0.501779,0.466192,0.192171,86,55,0.609929
phrases_claas,1,"[0, 1, 2, 3]",0.576512,0.576512,0.124555,142,20,0.876543
phrase_closeness,2,"[0, 1, 2, 3]",0.946619,0.758007,0.192171,219,47,0.823308


# New

In [46]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L_train)
pred_train =label_model.predict(L_train)
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=train_df, y=pred_train, L=L_train
)

In [47]:
df_train_filtered['noisy_labels'] =probs_train_filtered 

/Users/remzicelebi/opt/anaconda3/envs/snorkel-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
df_train_filtered.relation = df_train_filtered.relation.replace(RELATION_CLASSES)

/Users/remzicelebi/opt/anaconda3/envs/snorkel-env/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [49]:
df_train_filtered.rename(columns={'relation':'crowd_label'}, inplace=True)
df_train_filtered.to_csv('train_data.csv',index=False)

/Users/remzicelebi/opt/anaconda3/envs/snorkel-env/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [50]:
test_df['noisy_labels']= preds_dev

In [51]:
test_df['label']= Y_dev
test_df.rename(columns={'relation':'crowd_label'}, inplace=True)
test_df.to_csv('test_data.csv',index=False)

In [52]:
len(df_train_filtered)

38772